In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import kerastuner as kt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score

C:\Users\DIGIFIX\AppData\Local\Temp\ipykernel_13916\2008626586.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
train_DF = pd.read_csv("train2.csv")
val_DF = pd.read_csv("validation.csv")
test_DF = pd.read_csv("test.csv")

In [3]:
def convert_to_float_list(s):
    # Removemos los corchetes rectos (nose como más llamar a esas vainas)
    clean_str = s.replace('[', '').replace(']', '').strip()
    float_list = [float(x) for x in clean_str.split()]
    return np.array(float_list)

In [4]:
def getLabels(df, label_name):
    y = df[label_name].copy()
    return (y)

In [5]:
test_DF['vectorized_favorite_genres'] = test_DF['vectorized_favorite_genres'].apply(convert_to_float_list)
train_DF['vectorized_favorite_genres'] = train_DF['vectorized_favorite_genres'].apply(convert_to_float_list)
val_DF['vectorized_favorite_genres'] = val_DF['vectorized_favorite_genres'].apply(convert_to_float_list)

In [6]:
test_DF['vectorized_anime_genres'] = test_DF['vectorized_anime_genres'].apply(convert_to_float_list)
train_DF['vectorized_anime_genres'] = train_DF['vectorized_anime_genres'].apply(convert_to_float_list)
val_DF['vectorized_anime_genres'] = val_DF['vectorized_anime_genres'].apply(convert_to_float_list)

In [7]:
testArray = test_DF[['vectorized_favorite_genres','vectorized_anime_genres']].values
testArray.reshape(len(test_DF),2)
trainArray = train_DF[['vectorized_favorite_genres','vectorized_anime_genres']].values
trainArray.reshape(len(train_DF),2)
valArray = val_DF[['vectorized_favorite_genres','vectorized_anime_genres']].values
valArray.reshape(len(val_DF),2)

array([[array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
               0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 1., 1., 0., 0., 0., 0., 0., 0.])                               ,
        array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0.])                               ],
       [array([0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0.,
               1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
               0., 0., 0., 0., 0., 1., 0., 1., 0.])                               ,
        array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 1., 0.])                               ]

In [8]:
testArray.shape

(21858, 2)

In [9]:
trainArray.shape

(69945, 2)

In [10]:
valArray.shape

(17487, 2)

In [11]:
testArray = np.array([np.concatenate(row) for row in testArray])
trainArray = np.array([np.concatenate(row) for row in trainArray])
valArray = np.array([np.concatenate(row) for row in valArray])

In [12]:
testArray

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [13]:
testArray.shape

(21858, 86)

In [14]:
trainArray

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 1., 0.]])

In [15]:
trainArray.shape

(69945, 86)

In [16]:
valArray

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [17]:
valArray.shape

(17487, 86)

In [18]:
trainLabels = getLabels(train_DF,'opinion')
trainLabels = trainLabels.values
trainLabels = np.stack(trainLabels)
trainLabels

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [19]:
testLabels = getLabels(test_DF,'opinion')
testLabels = testLabels.values
testLabels = np.stack(testLabels)
testLabels

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [20]:
valLabels = getLabels(val_DF,'opinion')
valLabels = valLabels.values
valLabels = np.stack(valLabels)
valLabels

array([1, 0, 0, ..., 1, 1, 1], dtype=int64)

In [21]:
print("\nTest labels shape:", testLabels.shape)
print("\nTrain labels shape:", trainLabels.shape)
print("\nvalidation labels shape:", valLabels.shape)


Test labels shape: (21858,)

Train labels shape: (69945,)

validation labels shape: (17487,)


In [22]:
def model_builder(hp):
    # Defino el modelo
    model = keras.Sequential()

    # creamos la input layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    
    # agregamos capas profundas, pues se recomienda tener almenos 1 capa profunda para que el modelo quede más mamado
    num_layers = hp.Int("num_layers", 1, 20)
    for i in range(num_layers):
        units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32)
        activation = hp.Choice(f"activation_{i}", ["relu", "tanh"])
        model.add(layers.Dense(units, activation=activation))
    
    # Output layer, predecirá si le gusta o no el anime sacando un boolean 0 o 1
    model.add(layers.Dense(1, activation="sigmoid"))
    
    # tuneamos el learning rate, en internet recomiendan que sean estos valores de 0.01 o 0.001 o 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    # Compilamos el modelo y a rezar porque salga bien
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

In [23]:
#se pueden usar RandomSearch, BayesianOptimization o Hyperband de acuerdo a la documentación del hipertunner, investigando en internet, dicen que hyperband ahorra recursos pq detiene los modelos mas "malitos" temprano, por ende vamos a usar este para que no se explote ni se demore mucho el pc
tuner = kt.Hyperband(
    model_builder,
    objective = 'val_accuracy', 
    max_epochs = 10,
    factor = 3,
    directory = '../AnimeClassification-main',
    project_name = 'modelo_mamadisimo_y_tuneado'
)

In [24]:
# Ejecución del tuner para la búsqueda de hiperparámetros
tuner.search(
    trainArray, 
    trainLabels, 
    epochs = 10, 
    validation_data = (valArray, valLabels)
)

Trial 30 Complete [00h 08m 31s]
val_accuracy: 0.7154457569122314

Best val_accuracy So Far: 0.7352890968322754
Total elapsed time: 01h 01m 44s


In [25]:
#sacamos el modelo mas mamado que haya
bestModel = tuner.get_best_hyperparameters(num_trials=1)[0]

In [26]:
model = tuner.hypermodel.build(bestModel)

In [27]:
trainedModel = model.fit(
    trainArray,
    trainLabels,
    #se que parecen pocas epochs, pero luego de probar varias veces, 3 es en donde mejor sale
    epochs = 3,
    validation_data = (valArray, valLabels)
)

Epoch 1/3


2186/2186 [==============================] - 9s 3ms/step - loss: 0.5700 - accuracy: 0.7197 - val_loss: 0.5578 - val_accuracy: 0.7287
Epoch 2/3
2186/2186 [==============================] - 7s 3ms/step - loss: 0.5513 - accuracy: 0.7268 - val_loss: 0.5502 - val_accuracy: 0.7325
Epoch 3/3
2186/2186 [==============================] - 7s 3ms/step - loss: 0.5376 - accuracy: 0.7354 - val_loss: 0.5466 - val_accuracy: 0.7332


In [27]:
#a probarlo con testArray
opinionPrediction = model.predict(testArray).round(0)

684/684 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [28]:
opinionPrediction

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [29]:
testLabels

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [30]:
accuracy = accuracy_score(testLabels, opinionPrediction, normalize=True)
precision = precision_score(testLabels, opinionPrediction, average='micro')

In [31]:
print("\nAccuracy:",accuracy)
print("\nPrecision:",precision)


Accuracy: 0.693796321712874

Precision: 0.693796321712874


In [ ]:
#Implementación de grafo
